# Weaviate

This notebook covers how to get started with the Weaviate vector store.

>[Weaviate](https://weaviate.io/) is an open-source vector database. It allows you to store data objects and vector embeddings from your favorite ML-models, and scale seamlessly into billions of data objects.

This notebook shows how to use the functionality related to the `Weaviate` vector database.

`Weaviate` can be deployed in many different ways depending on your requirements. For example, you can either connect to a [Weaviate Cloud Services](https://console.weaviate.cloud) instance or a [local Docker instance](https://weaviate.io/developers/weaviate/installation/docker-compose). 
See the `Weaviate` [installation instructions](https://weaviate.io/developers/weaviate/installation) for more information.

## Installation

In [ ]:
# install package
!pip install -U langchain-weaviate

## Environment Setup

This example assumes you have a local weaviate instance setup and running on at `http://localhost:8080`.



We want to use `OpenAIEmbeddings` so we have to an Open AI API key and store it in the environment variable `OPENAI_API_KEY`.


## Usage

## Similarity Search

Below youm can see a minimal example of how to approach a simple similarity search.

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate